## 1. Setup & Install Dependencies

In [ ]:
# Install required packages
!pip install httpx feedparser pandas numpy matplotlib seaborn -q
print("✅ Dependencies installed!")

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
import asyncio
import httpx
import feedparser
import json
import warnings
warnings.filterwarnings('ignore')

# TensorFlow
import tensorflow as tf
from tensorflow.keras import layers, models, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU Available: {tf.config.list_physical_devices('GPU')}")

## 2. Kanyakumari Location Configuration

In [ ]:
# Kanyakumari coordinates
KANYAKUMARI_LAT = 8.0883
KANYAKUMARI_LON = 77.5385

print(f"📍 Monitoring Location: Kanyakumari, Tamil Nadu, India")
print(f"   Latitude: {KANYAKUMARI_LAT}°N")
print(f"   Longitude: {KANYAKUMARI_LON}°E")
print(f"   Region: Southernmost tip of Indian Peninsula")
print(f"   Seas: Arabian Sea, Bay of Bengal, Indian Ocean confluence")

## 3. Real-Time Data Fetcher

In [ ]:
class KanyakumariOceanMonitor:
    """Real-time ocean data fetcher for Kanyakumari region."""
    
    def __init__(self, lat: float = KANYAKUMARI_LAT, lon: float = KANYAKUMARI_LON):
        self.lat = lat
        self.lon = lon
        
    def fetch_marine_data(self) -> Dict:
        """Fetch current marine/wave data from Open-Meteo."""
        url = "https://marine-api.open-meteo.com/v1/marine"
        params = {
            "latitude": self.lat,
            "longitude": self.lon,
            "current": "wave_height,wave_direction,wave_period,swell_wave_height,swell_wave_direction,swell_wave_period",
            "hourly": "wave_height,wave_direction,wave_period",
            "forecast_days": 3,
            "timezone": "Asia/Kolkata"
        }
        
        try:
            response = httpx.get(url, params=params, timeout=30)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"⚠️ Marine data fetch error: {e}")
            return self._mock_marine_data()
    
    def fetch_weather_data(self) -> Dict:
        """Fetch weather data from Open-Meteo."""
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": self.lat,
            "longitude": self.lon,
            "current": "temperature_2m,relative_humidity_2m,pressure_msl,wind_speed_10m,wind_direction_10m",
            "hourly": "temperature_2m,pressure_msl,wind_speed_10m",
            "forecast_days": 3,
            "timezone": "Asia/Kolkata"
        }
        
        try:
            response = httpx.get(url, params=params, timeout=30)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"⚠️ Weather data fetch error: {e}")
            return {}
    
    def fetch_earthquakes(self, days: int = 7, min_magnitude: float = 4.0) -> List[Dict]:
        """Fetch recent earthquakes from USGS."""
        end_time = datetime.utcnow()
        start_time = end_time - timedelta(days=days)
        
        url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
        params = {
            "format": "geojson",
            "starttime": start_time.strftime("%Y-%m-%d"),
            "endtime": end_time.strftime("%Y-%m-%d"),
            "minmagnitude": min_magnitude,
            "minlatitude": -10,
            "maxlatitude": 30,
            "minlongitude": 60,
            "maxlongitude": 100,
            "orderby": "time"
        }
        
        try:
            response = httpx.get(url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()
            
            earthquakes = []
            for feature in data.get("features", [])[:10]:
                props = feature["properties"]
                coords = feature["geometry"]["coordinates"]
                earthquakes.append({
                    "magnitude": props.get("mag"),
                    "place": props.get("place"),
                    "time": datetime.fromtimestamp(props.get("time", 0) / 1000).isoformat(),
                    "depth_km": coords[2] if len(coords) > 2 else None,
                    "latitude": coords[1],
                    "longitude": coords[0],
                    "tsunami_flag": props.get("tsunami", 0)
                })
            return earthquakes
        except Exception as e:
            print(f"⚠️ Earthquake data fetch error: {e}")
            return []
    
    def _mock_marine_data(self) -> Dict:
        """Generate mock marine data for testing."""
        return {
            "current": {
                "wave_height": np.random.uniform(0.5, 2.5),
                "wave_direction": np.random.uniform(0, 360),
                "wave_period": np.random.uniform(4, 12),
                "swell_wave_height": np.random.uniform(0.3, 1.5),
                "swell_wave_direction": np.random.uniform(0, 360),
                "swell_wave_period": np.random.uniform(6, 15)
            },
            "hourly": {
                "time": [(datetime.now() + timedelta(hours=i)).isoformat() for i in range(72)],
                "wave_height": [np.random.uniform(0.5, 2.5) for _ in range(72)]
            }
        }
    
    def calculate_tsunami_risk(self, marine_data: Dict, earthquakes: List[Dict]) -> Dict:
        """Calculate tsunami risk score based on multiple factors."""
        risk_score = 0.0
        factors = []
        
        # Wave height factor
        current = marine_data.get("current", {})
        wave_height = current.get("wave_height", 0)
        if wave_height > 3.0:
            risk_score += 0.2
            factors.append(f"High waves: {wave_height:.1f}m")
        
        # Earthquake factor
        for eq in earthquakes[:5]:
            mag = eq.get("magnitude", 0)
            depth = eq.get("depth_km", 100)
            
            if mag >= 7.0 and depth < 70:
                risk_score += 0.4
                factors.append(f"Major earthquake: M{mag} at {depth}km depth")
            elif mag >= 6.0 and depth < 50:
                risk_score += 0.2
                factors.append(f"Significant earthquake: M{mag}")
            
            if eq.get("tsunami_flag", 0) == 1:
                risk_score += 0.3
                factors.append("Tsunami flag from USGS")
        
        # Determine risk level
        if risk_score >= 0.6:
            risk_level = "HIGH"
        elif risk_score >= 0.3:
            risk_level = "MODERATE"
        else:
            risk_level = "LOW"
        
        return {
            "risk_score": min(risk_score, 1.0),
            "risk_level": risk_level,
            "factors": factors if factors else ["Normal conditions"]
        }

# Initialize monitor
monitor = KanyakumariOceanMonitor()
print("✅ Ocean Monitor initialized!")

## 4. Fetch Real-Time Data

In [ ]:
# Fetch all data
print("🔄 Fetching real-time data from APIs...")
print("="*60)

marine_data = monitor.fetch_marine_data()
weather_data = monitor.fetch_weather_data()
earthquakes = monitor.fetch_earthquakes()
tsunami_risk = monitor.calculate_tsunami_risk(marine_data, earthquakes)

# Display current conditions
current_marine = marine_data.get("current", {})
current_weather = weather_data.get("current", {})

print(f"\n📊 CURRENT CONDITIONS - {datetime.now().strftime('%Y-%m-%d %H:%M:%S IST')}")
print("="*60)
print(f"\n🌊 MARINE DATA:")
print(f"   Wave Height: {current_marine.get('wave_height', 'N/A')} m")
print(f"   Wave Period: {current_marine.get('wave_period', 'N/A')} s")
print(f"   Wave Direction: {current_marine.get('wave_direction', 'N/A')}°")
print(f"   Swell Height: {current_marine.get('swell_wave_height', 'N/A')} m")

print(f"\n🌤️ WEATHER DATA:")
print(f"   Temperature: {current_weather.get('temperature_2m', 'N/A')}°C")
print(f"   Humidity: {current_weather.get('relative_humidity_2m', 'N/A')}%")
print(f"   Pressure: {current_weather.get('pressure_msl', 'N/A')} hPa")
print(f"   Wind Speed: {current_weather.get('wind_speed_10m', 'N/A')} km/h")

print(f"\n🔴 TSUNAMI RISK ASSESSMENT:")
print(f"   Risk Level: {tsunami_risk['risk_level']}")
print(f"   Risk Score: {tsunami_risk['risk_score']:.2f}")
print(f"   Factors: {', '.join(tsunami_risk['factors'])}")

print(f"\n🌍 RECENT EARTHQUAKES ({len(earthquakes)} found):")
for eq in earthquakes[:3]:
    print(f"   M{eq['magnitude']}: {eq['place']} ({eq['time'][:10]})")

print("\n" + "="*60)

## 5. Visualize Wave Forecast

In [ ]:
# Plot wave height forecast
hourly = marine_data.get("hourly", {})
times = hourly.get("time", [])[:48]  # Next 48 hours
wave_heights = hourly.get("wave_height", [])[:48]

if times and wave_heights:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('🌊 Kanyakumari Ocean Conditions - 48 Hour Forecast', fontsize=16, fontweight='bold')
    
    # Wave height over time
    ax1 = axes[0, 0]
    hours = range(len(wave_heights))
    ax1.fill_between(hours, wave_heights, alpha=0.3, color='blue')
    ax1.plot(hours, wave_heights, 'b-', linewidth=2)
    ax1.axhline(y=2.0, color='orange', linestyle='--', label='Moderate threshold')
    ax1.axhline(y=4.0, color='red', linestyle='--', label='High threshold')
    ax1.set_xlabel('Hours from now')
    ax1.set_ylabel('Wave Height (m)')
    ax1.set_title('Wave Height Forecast')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Wave height distribution
    ax2 = axes[0, 1]
    ax2.hist(wave_heights, bins=20, color='steelblue', edgecolor='white', alpha=0.7)
    ax2.axvline(x=np.mean(wave_heights), color='red', linestyle='--', label=f'Mean: {np.mean(wave_heights):.2f}m')
    ax2.set_xlabel('Wave Height (m)')
    ax2.set_ylabel('Frequency')
    ax2.set_title('Wave Height Distribution')
    ax2.legend()
    
    # Current conditions gauge
    ax3 = axes[1, 0]
    current_wave = current_marine.get('wave_height', 1.0)
    categories = ['Normal\n(0-1m)', 'Moderate\n(1-2m)', 'High\n(2-3m)', 'Extreme\n(3m+)']
    colors = ['green', 'yellow', 'orange', 'red']
    values = [1, 1, 1, 1]
    
    # Determine current category
    if current_wave < 1:
        highlight = 0
    elif current_wave < 2:
        highlight = 1
    elif current_wave < 3:
        highlight = 2
    else:
        highlight = 3
    
    bar_colors = ['lightgray'] * 4
    bar_colors[highlight] = colors[highlight]
    
    ax3.bar(categories, values, color=bar_colors, edgecolor='black')
    ax3.set_ylim(0, 1.5)
    ax3.set_title(f'Current Status: {current_wave:.1f}m')
    ax3.set_ylabel('Severity Level')
    
    # Risk assessment pie
    ax4 = axes[1, 1]
    risk_score = tsunami_risk['risk_score']
    safe_score = 1 - risk_score
    
    risk_colors = ['green' if risk_score < 0.3 else 'orange' if risk_score < 0.6 else 'red', 'lightgray']
    ax4.pie([risk_score, safe_score], labels=['Risk', 'Safe'], colors=risk_colors,
            autopct='%1.1f%%', startangle=90, explode=(0.05, 0))
    ax4.set_title(f'Tsunami Risk: {tsunami_risk["risk_level"]}')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No forecast data available for visualization")

## 6. CNN-LSTM Hybrid Model Architecture

In [ ]:
class AttentionLayer(layers.Layer):
    """Custom attention layer for sequence modeling."""
    
    def __init__(self, units: int = 64, **kwargs):
        super().__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.W = self.add_weight(
            name='attention_weight',
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform',
            trainable=True
        )
        self.b = self.add_weight(
            name='attention_bias',
            shape=(self.units,),
            initializer='zeros',
            trainable=True
        )
        self.u = self.add_weight(
            name='attention_context',
            shape=(self.units, 1),
            initializer='glorot_uniform',
            trainable=True
        )
        super().build(input_shape)

    def call(self, inputs):
        score = tf.tanh(tf.tensordot(inputs, self.W, axes=1) + self.b)
        attention_weights = tf.nn.softmax(tf.tensordot(score, self.u, axes=1), axis=1)
        context = tf.reduce_sum(inputs * attention_weights, axis=1)
        return context

    def get_config(self):
        config = super().get_config()
        config.update({'units': self.units})
        return config

print("✅ AttentionLayer defined!")

In [ ]:
def build_cnn_backbone(image_shape=(64, 64, 3)):
    """Build CNN backbone for spatial feature extraction."""
    inputs = layers.Input(shape=image_shape, name="image_input")
    
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    
    return Model(inputs, x, name="cnn_backbone")


def build_lstm_backbone(seq_len=24, seq_features=8):
    """Build LSTM backbone for temporal sequence modeling."""
    inputs = layers.Input(shape=(seq_len, seq_features), name="sequence_input")
    
    x = layers.Masking(mask_value=0.0)(inputs)
    
    x = layers.Bidirectional(
        layers.LSTM(128, return_sequences=True, dropout=0.2)
    )(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Bidirectional(
        layers.LSTM(64, return_sequences=True, dropout=0.2)
    )(x)
    
    # Apply attention
    x = AttentionLayer(units=64, name="attention")(x)
    x = layers.Dropout(0.3)(x)
    
    return Model(inputs, x, name="lstm_backbone")


def build_multimodal_model(
    image_shape=(64, 64, 3),
    seq_len=24,
    seq_features=8,
    num_wave_classes=4,
    num_tsunami_classes=3
):
    """Build complete multimodal CNN-LSTM hybrid model."""
    
    # Build backbones
    cnn = build_cnn_backbone(image_shape)
    lstm = build_lstm_backbone(seq_len, seq_features)
    
    # Get inputs and features
    image_input = cnn.input
    image_features = cnn.output
    
    seq_input = lstm.input
    seq_features_out = lstm.output
    
    # Fusion layer
    fused = layers.Concatenate(name="multimodal_fusion")([image_features, seq_features_out])
    
    # Shared dense layers
    x = layers.Dense(256, activation='relu')(fused)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    # Output heads
    wave_output = layers.Dense(num_wave_classes, activation='softmax', name='wave_severity')(x)
    tsunami_output = layers.Dense(num_tsunami_classes, activation='softmax', name='tsunami_risk')(x)
    height_output = layers.Dense(1, activation='linear', name='wave_height_meters')(x)
    
    model = Model(
        inputs=[image_input, seq_input],
        outputs=[wave_output, tsunami_output, height_output],
        name="multimodal_cnn_lstm_hybrid"
    )
    
    return model

print("✅ Model builder functions defined!")

In [ ]:
# Build and compile the model
model = build_multimodal_model()

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'wave_severity': 'categorical_crossentropy',
        'tsunami_risk': 'categorical_crossentropy',
        'wave_height_meters': 'mse'
    },
    loss_weights={
        'wave_severity': 1.0,
        'tsunami_risk': 1.5,
        'wave_height_meters': 0.5
    },
    metrics={
        'wave_severity': ['accuracy'],
        'tsunami_risk': ['accuracy'],
        'wave_height_meters': ['mae']
    }
)

print("\n" + "="*80)
print("MULTIMODAL CNN-LSTM HYBRID MODEL FOR OCEAN WAVE & TSUNAMI PREDICTION")
print("="*80)
model.summary()

## 7. Generate Synthetic Training Data

In [ ]:
def generate_synthetic_data(n_samples=1000, seq_len=24, seq_features=8):
    """Generate synthetic training data for demonstration."""
    np.random.seed(42)
    
    # Image data (simulating satellite/heatmap data)
    images = np.random.randn(n_samples, 64, 64, 3).astype(np.float32)
    
    # Sequence data (wave height, pressure, wind, etc.)
    sequences = np.random.randn(n_samples, seq_len, seq_features).astype(np.float32)
    
    # Labels
    wave_severity = np.random.randint(0, 4, n_samples)
    wave_severity_onehot = tf.keras.utils.to_categorical(wave_severity, 4)
    
    tsunami_risk = np.random.randint(0, 3, n_samples)
    tsunami_risk_onehot = tf.keras.utils.to_categorical(tsunami_risk, 3)
    
    wave_height = np.random.uniform(0.5, 5.0, n_samples).astype(np.float32)
    
    return {
        'images': images,
        'sequences': sequences,
        'wave_severity': wave_severity_onehot,
        'tsunami_risk': tsunami_risk_onehot,
        'wave_height': wave_height
    }

# Generate data
print("🔄 Generating synthetic training data...")
data = generate_synthetic_data(n_samples=2000)

print(f"✅ Generated {len(data['images'])} samples")
print(f"   Images shape: {data['images'].shape}")
print(f"   Sequences shape: {data['sequences'].shape}")
print(f"   Wave severity classes: {data['wave_severity'].shape}")
print(f"   Tsunami risk classes: {data['tsunami_risk'].shape}")

## 8. Train the Model

In [ ]:
# Split data
split_idx = int(len(data['images']) * 0.8)

train_images = data['images'][:split_idx]
train_sequences = data['sequences'][:split_idx]
train_wave_severity = data['wave_severity'][:split_idx]
train_tsunami_risk = data['tsunami_risk'][:split_idx]
train_wave_height = data['wave_height'][:split_idx]

val_images = data['images'][split_idx:]
val_sequences = data['sequences'][split_idx:]
val_wave_severity = data['wave_severity'][split_idx:]
val_tsunami_risk = data['tsunami_risk'][split_idx:]
val_wave_height = data['wave_height'][split_idx:]

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")

In [ ]:
# Training callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
]

# Train the model
print("\n🚀 Starting training...")
print("="*60)

history = model.fit(
    [train_images, train_sequences],
    {
        'wave_severity': train_wave_severity,
        'tsunami_risk': train_tsunami_risk,
        'wave_height_meters': train_wave_height
    },
    validation_data=(
        [val_images, val_sequences],
        {
            'wave_severity': val_wave_severity,
            'tsunami_risk': val_tsunami_risk,
            'wave_height_meters': val_wave_height
        }
    ),
    epochs=20,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Training complete!")

## 9. Training History Visualization

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Model Training History', fontsize=16, fontweight='bold')

# Total loss
ax1 = axes[0, 0]
ax1.plot(history.history['loss'], label='Training Loss')
ax1.plot(history.history['val_loss'], label='Validation Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Total Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Wave severity accuracy
ax2 = axes[0, 1]
ax2.plot(history.history['wave_severity_accuracy'], label='Training')
ax2.plot(history.history['val_wave_severity_accuracy'], label='Validation')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Wave Severity Classification Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Tsunami risk accuracy
ax3 = axes[1, 0]
ax3.plot(history.history['tsunami_risk_accuracy'], label='Training')
ax3.plot(history.history['val_tsunami_risk_accuracy'], label='Validation')
ax3.set_xlabel('Epoch')
ax3.set_ylabel('Accuracy')
ax3.set_title('Tsunami Risk Classification Accuracy')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Wave height MAE
ax4 = axes[1, 1]
ax4.plot(history.history['wave_height_meters_mae'], label='Training')
ax4.plot(history.history['val_wave_height_meters_mae'], label='Validation')
ax4.set_xlabel('Epoch')
ax4.set_ylabel('MAE (meters)')
ax4.set_title('Wave Height Prediction MAE')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Make Predictions with Real Data

In [ ]:
def prepare_realtime_input(marine_data: Dict, weather_data: Dict, seq_len: int = 24):
    """Prepare real-time data for model input."""
    
    # Create synthetic image (placeholder for satellite data)
    image = np.random.randn(1, 64, 64, 3).astype(np.float32)
    
    # Create sequence from hourly data
    hourly_marine = marine_data.get("hourly", {})
    hourly_weather = weather_data.get("hourly", {})
    
    wave_heights = hourly_marine.get("wave_height", [1.0] * seq_len)[:seq_len]
    wave_directions = hourly_marine.get("wave_direction", [180] * seq_len)[:seq_len]
    wave_periods = hourly_marine.get("wave_period", [6] * seq_len)[:seq_len]
    
    temperatures = hourly_weather.get("temperature_2m", [28] * seq_len)[:seq_len]
    pressures = hourly_weather.get("pressure_msl", [1013] * seq_len)[:seq_len]
    wind_speeds = hourly_weather.get("wind_speed_10m", [15] * seq_len)[:seq_len]
    
    # Pad if needed
    def pad_list(lst, length, default=0):
        lst = list(lst) if lst else [default] * length
        return lst + [default] * (length - len(lst)) if len(lst) < length else lst[:length]
    
    wave_heights = pad_list(wave_heights, seq_len, 1.0)
    wave_directions = pad_list(wave_directions, seq_len, 180)
    wave_periods = pad_list(wave_periods, seq_len, 6)
    temperatures = pad_list(temperatures, seq_len, 28)
    pressures = pad_list(pressures, seq_len, 1013)
    wind_speeds = pad_list(wind_speeds, seq_len, 15)
    
    # Additional features
    humidity = [70] * seq_len
    visibility = [10] * seq_len
    
    # Stack into sequence
    sequence = np.array([
        wave_heights,
        wave_directions,
        wave_periods,
        temperatures,
        pressures,
        wind_speeds,
        humidity,
        visibility
    ]).T.astype(np.float32)
    
    # Normalize
    sequence = (sequence - sequence.mean(axis=0)) / (sequence.std(axis=0) + 1e-8)
    
    return image, sequence.reshape(1, seq_len, 8)

# Prepare input from real data
image_input, seq_input = prepare_realtime_input(marine_data, weather_data)

print(f"Image input shape: {image_input.shape}")
print(f"Sequence input shape: {seq_input.shape}")

In [ ]:
# Make prediction
predictions = model.predict([image_input, seq_input], verbose=0)

wave_probs = predictions[0][0]
tsunami_probs = predictions[1][0]
wave_height_pred = predictions[2][0][0]

WAVE_CLASSES = ['NORMAL', 'MODERATE', 'HIGH', 'EXTREME']
TSUNAMI_CLASSES = ['NONE', 'LOW', 'HIGH']

wave_class = WAVE_CLASSES[np.argmax(wave_probs)]
tsunami_class = TSUNAMI_CLASSES[np.argmax(tsunami_probs)]

print("\n" + "="*60)
print("🤖 AI PREDICTION FOR KANYAKUMARI")
print("="*60)

print(f"\n🌊 WAVE SEVERITY: {wave_class}")
print(f"   Probabilities:")
for i, cls in enumerate(WAVE_CLASSES):
    bar = '█' * int(wave_probs[i] * 20)
    print(f"     {cls:10s}: {wave_probs[i]*100:5.1f}% {bar}")

print(f"\n🔴 TSUNAMI RISK: {tsunami_class}")
print(f"   Probabilities:")
for i, cls in enumerate(TSUNAMI_CLASSES):
    bar = '█' * int(tsunami_probs[i] * 20)
    print(f"     {cls:10s}: {tsunami_probs[i]*100:5.1f}% {bar}")

print(f"\n📏 PREDICTED WAVE HEIGHT: {wave_height_pred:.2f} meters")
print(f"   (Current actual: {current_marine.get('wave_height', 'N/A')} meters)")

print("\n" + "="*60)

## 11. Save the Model

In [ ]:
# Save the model
model.save('kanyakumari_ocean_model.keras')
print("✅ Model saved as 'kanyakumari_ocean_model.keras'")

# Download link for Colab
try:
    from google.colab import files
    files.download('kanyakumari_ocean_model.keras')
except:
    print("(Not in Colab environment - model saved locally)")

## 12. Summary & Next Steps

### What We Built:
1. **Real-time Data Fetcher** - Fetches live ocean and weather data for Kanyakumari
2. **CNN-LSTM Hybrid Model** - Multimodal architecture with attention mechanism
3. **Multi-task Predictions** - Wave severity, tsunami risk, and wave height

### To Improve:
- Use real satellite imagery instead of synthetic data
- Train on historical ocean event data
- Add more seismic features for better tsunami prediction
- Deploy as a web service using FastAPI

### APIs Used:
- **Open-Meteo Marine API** - Wave data
- **Open-Meteo Weather API** - Weather conditions
- **USGS Earthquake API** - Seismic monitoring

---
**📍 Kanyakumari Ocean Wave & Tsunami Prediction System**